In [1]:
import openpyxl
import os
import sys
import time
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import numpy as np

In [2]:
import pandas as pd

In [69]:
df = pd.read_excel("C:/Users/ADMIN/OneDrive/바탕 화면/토이프로젝트/맛집리스트.xlsx")
#df

In [5]:
data = df['추천맛집'].to_list()

# 1. 체인점 거르기
- 수집한 설문 답변이 체인점인 식당도 있고, 개인식당도 존재
- 체인점의 경우 어떤 지점이 맛집인지 정확한 구분을 위해 서울 내 평점 가장 높은 지점의 상호명으로 데이터 재수집

In [141]:
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(executable_path='C:/Users/user/Downloads/chromedriver_win32/chromedriver.exe')

C:\Users\user\AppData\Local\Temp\ipykernel_22276\1285491379.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='C:/Users/user/Downloads/chromedriver_win32/chromedriver.exe')


In [142]:
체인_list = []
평점_list = []
체인_평점 = []
체인_최종 = []
식당_list = []
검색x_네이버 = []
체인x_list = []

for data in data :
    
    time.sleep(2)
        
        #음식점 상호명 + 서울음식점
    driver.get(f"https://m.map.naver.com/search2/search.naver?query={data}서울음식점&sm=hty&style=v5")
        #dirver.implicitly_wait(time_to_wait=3)
        
    # 차단막는 코드, 랜덤으로 time.sleep 지정
    seed = np.random.randint(100)
    np.random.seed(seed)
    a = np.random.randint(2)
    time.sleep(a)

    for i in range(5) :
        driver.execute_script('scrollBy(0,2000)')
        time.sleep(1)
    
    elements = driver.find_elements(By.XPATH, '//*[@id="ct"]/div[2]/ul/li')
    
    if len(elements) > 1:
        for i,name in enumerate(elements) :
            el_name = elements[i].find_element(By.CSS_SELECTOR, f'#ct > div.search_listview._content._ctList > ul > li:nth-child({i+1}) > div.item_info > a.a_item.a_item_distance._linkSiteview > div > strong').text
            try :
                if '점' in el_name and data == el_name.split()[0]:
                    체인_list.append(el_name)
                elif '점' not in el_name and data == el_name :
                    체인x_list.append(el_name)

            except Exception :
                pass
            
            
        if len(체인_list) != 0 :
            for name in 체인_list :
                try :
                    driver.get(f"https://m.map.naver.com/search2/search.naver?query={name}&sm=hty&style=v5")
                    time.sleep(2)
                    driver.find_element(By.XPATH, '//*[@id="ct"]/div[2]/ul/li/div[1]/a/div').click()
                    time.sleep(2)
                    평점_list.append(driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/em').text)
                    time.sleep(1)
                    식당_list.append(name)

                except Exception :
                    pass
            try : 
                체인_평점.append(max(평점_list))
                idx = 평점_list.index(max(평점_list))
                체인_최종.append(식당_list[idx])
                time.sleep(1)
                평점_list = []
                식당_list = []
                체인_list = []
            
            except Exception :
                평점_list = []
                식당_list = []
                체인_list = []
                pass
        
        else :
            pass

    elif len(elements) == 1 :
        체인x_list.append(data)
      
    else :
        검색x_네이버.append(data)
        pass
    

 #맨 마지막에 drop_duplicates 사용해서 중복 제거하기    

In [151]:
#데이터 셋 구축

#체인X_list 중복제거 후 데이터 프레임 생성
체인x = pd.DataFrame(체인x_list).rename(columns={0 : '음식점명'}).drop_duplicates(subset='음식점명', keep='first').reset_index().drop(columns='index')

#체인 맛집 데이터 프레임 생성
#추천 맛집 컬럼 : 추후 기존의 df 데이터 프레임과 join을 수행하기 위해 새롭게 생성
체인o = pd.DataFrame(체인_최종).rename(columns={0:'음식점명'}).reset_index().drop(columns='index')
음식점명2 = 체인o['음식점명'].str.split(' ')
체인o['추천맛집'] = 음식점명2.str[0]

#oo서울음식점으로 검색이 불가능한 데이터
검색x_네이버 = pd.DataFrame(검색x_네이버).rename(columns={0:'음식점명'}).reset_index().drop(columns='index')

#위 3개의 데이터 프레임 결합 
df_최종 = pd.concat([체인x, 체인o, 검색x_네이버], axis=0).reset_index().drop(columns='index')
#추천 맛집 컬럼 결측치 처리 : 음식점명으로 대치
df_최종['추천맛집'].fillna(df_최종['음식점명'], inplace=True)
df_최종.drop_duplicates(subset=['추천맛집'], keep='last', inplace=True)


#df_최종 & 기존의 df 데이터 셋 '추천 맛집' 컬럼을 기준으로 join
df_최종 = pd.merge(df_최종, df, how='left', on='추천맛집').drop(columns='추천맛집')

#엑셀로 내보내기
df_최종.to_excel("C:/Users/user/Downloads/맛집리스트_v0.2.xlsx", index=False, encoding='utf-8')

# 2. 네이버지도 모바일 크롤링
- 리뷰, 평점, 키워드 리뷰 모두 네이버맵에서 크롤링

In [ ]:
df = pd.read_csv('C:/Users/user/Downloads/맛집리스트_v0.2.xlsx')
data = df['추천맛집'].to_list()

In [58]:
# 드라이버 실행
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(executable_path='C:/Users/ADMIN/OneDrive/바탕 화면/chromedriver_win32/chromedriver.exe') #404 not found 오류 방지 

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_3812\1392209137.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='C:/Users/ADMIN/OneDrive/바탕 화면/chromedriver_win32/chromedriver.exe') #404 not found 오류 방지


In [59]:
검색불가_네이버 = []
평점_네이버 = []
식당명_네이버 = []
식당별_키워드_네이버 = []
음식점_카테고리 = []

for i in data:
    driver.get(f"https://m.map.naver.com/search2/search.naver?query={i}&sm=hty&style=v5")
    driver.implicitly_wait(time_to_wait=3)
    
    # 검색결과 조건문
    try:
        # 상세페이지 진입
        driver.implicitly_wait(time_to_wait=2)
        driver.find_element(By.XPATH, '//*[@id="ct"]/div[2]/ul/li[1]/div[1]/a/div/strong').click()
        time.sleep(3)
        
        # 식당명 저장
        식당명_네이버.append(i)
        time.sleep(3)

        # 평점 없는 경우의 조건문
        try:
            평점_네이버.append(driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/em').text)
            time.sleep(2)

        except Exception:
            평점_네이버.append('x')
            time.sleep(2)
            pass
        
        #음식점 카테고리
        음식점_카테고리.append(driver.find_element(By.XPATH, '//*[@id="_title"]/span[2]').text)
        time.sleep(2)
        
         # '사진' 클릭
        사진_index = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div').text.split(' ').index('사진') + 1   
        driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[{사진_index}]/span').click() 
        driver.implicitly_wait(time_to_wait=5)
        
        # 내부사진 파일 저장
        내부_index = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div[2]/div/div/div/div').text.split('\n').index('내부')+1 # 내부 클릭
        driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[7]/div[2]/div/div/div/div/div[{내부_index}]/a').click()
        time.sleep(2)
        내부_url = driver.find_element(By.XPATH, '//*[@id="내부_0"]').get_attribute('src')    # scr속성이 url링크
        urllib.request.urlretrieve(내부_url, f"C:/Users/Juns/Desktop/크롤링 이미지/{i}_내부사진.png")    # 기존에 만둘어둔 폴더 경로에 저장
        
        # 대표음식사진 파일 저장
        음식_index = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div[2]/div/div/div/div').text.split('\n').index('방문자사진')+1
        driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[7]/div[2]/div/div/div/div/div[{음식_index}]/a').click()
        time.sleep(2)
        음식_url = driver.find_element(By.XPATH, '//*[@id="visitor_0"]').get_attribute('src')   
        urllib.request.urlretrieve(음식_url, f"C:/Users/Juns/Desktop/크롤링 이미지/{i}_대표음식.png")  
        
        # '리뷰' 클릭
        리뷰_index = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div').text.split(' ').index('리뷰') + 1    # '리뷰'인덱스 추출
        driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[{리뷰_index}]/span').click()   # 인덱스번호는 0부터 시작하기 때문에 +1
        driver.implicitly_wait(time_to_wait=5)
        driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[{리뷰_index}]/span').click()   # 한번에 안눌리는 경우로인해 두번 클릭
        time.sleep(3)
        
        try:
            # 키워드 저장
            키워드_네이버 = []    # 키워드 리스트는 데이터프레임 저장을 위해 계속 갱신되어야함
            for z in range(1,6):
                time.sleep(1)
                키워드_네이버.append(driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[2]/ul/li[{z}]/div[2]/span[1]').text)
            식당별_키워드_네이버.append(키워드_네이버)  
        
        except Exception :
            식당별_키워드_네이버.append('x')
            time.sleep(1)
        
    
    # 에외처리 (검색불가)        
    except Exception:
        검색불가_네이버.append(i)
        driver.switch_to.window(driver.window_handles[0]);
        pass
    
driver.close()

In [60]:
len(식당명_네이버)

109

In [61]:
len(평점_네이버)

109

In [63]:
len(식당별_키워드_네이버)

109

In [65]:
검색불가_네이버

[]

In [2]:
columns=['키워드1','키워드2','키워드3','키워드4','키워드5']
맛집_list = pd.DataFrame(식당별_키워드_네이버, index=식당명_네이버, columns=columns)
맛집_list['평점_네이버'] = 평점_네이버 
맛집_list

In [1]:
맛집_list = pd.DataFrame(맛집_list).reset_index().rename(columns={'index' : '음식점명'})
맛집_list

In [68]:
맛집_list.to_excel("C:/Users/ADMIN/OneDrive/바탕 화면/토이프로젝트/맛집리스트_v0.3.xlsx", index=False, encoding='utf-8')